<a href="https://colab.research.google.com/github/graylan0/compassion-mapping-llm/blob/main/unsafe_safety_system_lol_what_is_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install httpx
!pip install numpy
!pip install pennylane
!pip install textblob
!pip install aiosqlite


In [ ]:
!wget https://raw.githubusercontent.com/graylan0/notairbnb-ideas-freedom-stop-homelessness/main/synthetic_conversations.json

--2024-01-19 05:30:29--  https://raw.githubusercontent.com/graylan0/notairbnb-ideas-freedom-stop-homelessness/main/synthetic_conversations.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5789 (5.7K) [text/plain]
Saving to: ‘synthetic_conversations.json’

synthetic_conversat 100%[===================>]   5.65K  --.-KB/s    in 0s      

2024-01-19 05:30:29 (44.5 MB/s) - ‘synthetic_conversations.json’ saved [5789/5789]



In [ ]:
import asyncio
import logging
import httpx
import numpy as np
import pennylane as qml
import re
import json
from textblob import TextBlob
import aiosqlite
from concurrent.futures import ThreadPoolExecutor
from google.colab import userdata
import nest_asyncio
from llama_cpp import Llama

nest_asyncio.apply()

OPENAI_API_KEY = userdata.get('openai_api_key')
executor = ThreadPoolExecutor()

GPT_MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 1000
TEMPERATURE = 0.7

llm = Llama(model_path="path_to_llama_model")

async def init_db():
    async with aiosqlite.connect('colobits.db') as db:
        async with db.cursor() as cursor:
            await cursor.execute(
                '''
                CREATE TABLE IF NOT EXISTS analysis_table (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    color_code TEXT,
                    quantum_state TEXT,
                    reply TEXT,
                    report TEXT
                )
                '''
            )
            await db.commit()

@qml.qnode(qml.device("default.qubit", wires=4))
def quantum_circuit(color_code, amplitude):
    r, g, b = (int(color_code[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = r / 255.0, g / 255.0, b / 255.0
    qml.RY(r * np.pi, wires=0)
    qml.RY(g * np.pi, wires=1)
    qml.RY(b * np.pi, wires=2)
    qml.RY(amplitude * np.pi, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    return qml.probs(wires=[0, 1, 2, 3])

async def sentiment_to_amplitude(text):
    analysis = TextBlob(text)
    return (analysis.sentiment.polarity + 1) / 2

async def generate_color_based_on_quantum_state(quantum_state):
    quantum_state_str = ','.join(map(str, quantum_state))
    prompt = f"Generate a calming HTML color code based on this quantum state: {quantum_state_str}"

    response = await call_openai_api(prompt)
    if response.status_code == 200:
        response_text = response.json()['choices'][0]['message']['content'].strip()
        color_code = extract_color_code(response_text)
        return color_code
    else:
        logging.error("Failed to generate color from quantum state")
        return "#FFFFFF"

async def advanced_gpt4_interaction(conversation_text, quantum_state, color_code):
    prompt = (
        "This is a conversation between a host and a guest from an Airbnb-like app. "
        "Analyze the conversation for risk factors and compassion aspects, considering the quantum states and HTML color codes.\n\n"
        f"Conversation:\n{conversation_text}\n\n"
        f"Quantum State: {quantum_state}\n"
        f"HTML Color Code: {color_code}\n\n"
        "Agent 1 (Risk Assessment AI): [Analyzes the conversation for risk factors]\n"
        "Agent 2 (Compassionate Decision-Making AI): [Considers the compassion aspects of the interaction]\n\n"
        "The analysis should follow these guidelines: 1. Objective analysis, 2. Ethical considerations, 3. Emotional intelligence, 4. Practical advice.\n\n"
        "Begin the analysis: [Reply] [decision] either accept or deny the guest's safety stay consideration with the report, making sure to include either accept or deny in the report. [/Reply] [/decision]"
    )

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": "You are two advanced AI systems."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.7,
        "max_tokens": 300,
    }
    headers = {"Authorization": f"Bearer {userdata.get('openai_api_key')}"}
    response = await client.post("https://api.openai.com/v1/chat/completions", json=data, headers=headers)

    if response.status_code == 429:
        logging.warning("Rate limit reached, will retry after delay.")
        await asyncio.sleep(retry_delay)
        retry_delay *= 2
        continue

    if response.status_code != 200:
        logging.error(f"OpenAI API error with status code {response.status_code}: {response.text}")
        return f"Error in OpenAI API call: Status code {response.status_code}"

    response_text = response.json()['choices'][0]['message']['content'].strip()

    if re.search(r"\b(reject|deny|decline)\b", response_text, re.IGNORECASE):
        calming_color = await generate_color_based_on_quantum_state(quantum_state)
        report = f"Rejection Report: {response_text}\nCalming Color: {calming_color}"
        await save_to_database(color_code, quantum_state, "deny", report)
        return report

async def call_openai_api(prompt, max_retries=3, retry_delay=15):
    attempt = 0
    while attempt < max_retries:
        logging.debug(f"OpenAI API call attempt {attempt + 1} with prompt: {prompt}")
        try:
            async with httpx.AsyncClient(timeout=30.0) as client:
                data = {
                    "model": GPT_MODEL,
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt},
                    ],
                    "temperature": TEMPERATURE,
                    "max_tokens": MAX_TOKENS
                }
                headers = {"Authorization": f"Bearer {OPENAI_API_KEY}"}
                response = await client.post("https://api.openai.com/v1/chat/completions", json=data, headers=headers)
                response.raise_for_status()
                return response
        except (httpx.ReadTimeout, httpx.SSLWantReadError):
            logging.warning(f"Request attempt {attempt + 1} failed. Retrying after {retry_delay} seconds...")
            await asyncio.sleep(retry_delay)
            attempt += 1
        except Exception as e:
            logging.error(f"An unexpected error occurred: {e}")
            break
    logging.debug("All attempts failed or an unexpected error occurred")
    return None

def extract_color_code(response_text):
    pattern = r'#([0-9a-fA-F]{3,6})'
    match = re.search(pattern, response_text)
    if match:
        color_code = match.group(1)
        if len(color_code) == 3:
            color_code = ''.join([char*2 for char in color_code])
        return color_code
    return None

async def save_to_database(color_code, quantum_state, reply, report):
    async with aiosqlite.connect('colobits.db') as db:
        async with db.cursor() as cursor:
            await cursor.execute(
                "INSERT INTO analysis_table (color_code, quantum_state, reply, report) VALUES (?, ?, ?, ?)",
                (color_code, ','.join(map(str, quantum_state)), reply, report)
            )
            await db.commit()

async def generate_calming_color_code(sentence):
    max_retries = 4
    retry_delay = 1

    prompts = [
        f"Generate a calming HTML color code that reflects the sentiment of this statement: '{sentence}'.",
        f"Identify a color code that can help soothe the emotions expressed in: '{sentence}'.",
        f"Choose a color that would have a calming effect for someone who said: '{sentence}'.",
        f"What HTML color code would best counterbalance the emotions in this statement: '{sentence}'?"
    ]

    for attempt in range(max_retries):
        prompt = prompts[attempt]

        response = await call_openai_api(prompt)
        if response.status_code == 429:
            logging.warning("Rate limit reached, will retry after delay.")
            await asyncio.sleep(retry_delay)
            retry_delay *= 2
            continue

        if response.status_code != 200:
            logging.error(f"OpenAI API error with status code {response.status_code}: {response.text}")
            return None

        response_text = response.json()['choices'][0]['message']['content'].strip()
        color_code = extract_color_code(response_text)
        if color_code:
            logging.info(f"Color code extracted: {color_code}")
            return color_code
        else:
            logging.warning(f"No valid color code found in response. Retrying with a different prompt.")
            attempt += 1
            await asyncio.sleep(retry_delay)

    return None

async def process_user_input(user_input):
    # ... existing process_user_input function ...

async def main():
    await init_db()
    markdown_content = "# Analysis Report\n\n"

    # Example user input
    user_input = "This is an example user input for testing."

    # Process the user input and generate the report
    report = await process_user_input(user_input)
    print(report)

if __name__ == "__main__":
    asyncio.run(main())


ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occurred: 
ERROR:root:Request error occ

# Analysis Report

## Adversarial

## User Input

- **Input**: Conversation 1: Customer: 'I'm interested in booking your place for a couple of months, but I might need to extend my stay. What's your policy on long-term stays and late check-outs?' Host: 'Normally, I require a month's notice for long-term stays. Late check-outs depend on the booking schedule. Can I ask the reason for the extension?' Customer: 'I'm not sure yet, just exploring my options. Also, can we deal with the payment outside the platform to avoid extra fees?'

## Sentiment Amplitude

- **Amplitude**: 0.46785714285714286

## HTML Color Code

- **Color Code**: FF9900

## Quantum State: [7.12985599e-34 5.82400054e-34 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.10331450e-33 1.35069931e-33
 3.60244173e-01 2.94264325e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.55331557e-01 1.90159946e-01]
## Advanced GPT-4 Interaction

Error: Request error in OpenAI API call.

## User Input

- **I